# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

### Imports

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import math as math
import seaborn as sns
import itertools
from scipy import stats
import matplotlib.pyplot as plt

### Lendo as tabelas

In [2]:
leitura = pd.ExcelFile('tweets_Nike.xlsx')
at = pd.read_excel(leitura, 'Treinamento')
aT = pd.read_excel(leitura, 'Teste')

### Lipando a tabela Treinamento

Para poder calcular a probabilidade dos tweets serem relevantes ou irrelevantes, é necessário limpar os tweets, para assim ter uma melhor leitura. Nesta célula, estamos limpando os tweets da tabela Treinamento.

In [3]:
at.Treinamento = at.Treinamento.str.replace(':', '', regex=False)
at.Treinamento = at.Treinamento.str.replace('@', '', regex=False)
at.Treinamento = at.Treinamento.str.replace('*', '', regex=False)
at.Treinamento = at.Treinamento.str.replace('“', '', regex=False)
at.Treinamento = at.Treinamento.str.replace('.', '', regex=False)
at.Treinamento = at.Treinamento.str.replace(')', '', regex=False)
at.Treinamento = at.Treinamento.str.replace('(', '', regex=False)
at.Treinamento = at.Treinamento.str.replace('_', '', regex=False)
at.Treinamento = at.Treinamento.str.replace(';', '', regex=False)
at.Treinamento = at.Treinamento.str.replace(',', '', regex=False)
at.Treinamento = at.Treinamento.str.replace('&', '', regex=False)
at.Treinamento = at.Treinamento.str.replace('/', '', regex=False)
at.Treinamento = at.Treinamento.str.replace('#', '', regex=False)
at.Treinamento = at.Treinamento.str.replace('%', '', regex=False)
at.Treinamento = at.Treinamento.str.replace('$', '', regex=False)
at.Treinamento = at.Treinamento.str.replace("!", '', regex=False)
at.Treinamento = at.Treinamento.str.replace("?", '', regex=False)
at.Treinamento = at.Treinamento.str.replace("-", '', regex=False)
at.Treinamento = at.Treinamento.str.replace("+", '', regex=False)
at.Treinamento = at.Treinamento.str.replace("=", '', regex=False)
at.Treinamento = at.Treinamento.str.replace('rt', '', regex=False)
at.Treinamento = at.Treinamento.str.replace("\n", '', regex=False)

### Limpando a tabela Teste

Também faremos uma limpeza nos tweets da tabela Teste, da mesma forma que fizemos com os tweets da tabela Treinamento.

In [4]:
aT.Teste = aT.Teste.str.replace(':', '', regex=False)
aT.Teste= aT.Teste.str.replace('@', '', regex=False)
aT.Teste = aT.Teste.str.replace('*', '', regex=False)
aT.Teste = aT.Teste.str.replace('“', '', regex=False)
aT.Teste = aT.Teste.str.replace('.', '', regex=False)
aT.Teste = aT.Teste.str.replace(')', '', regex=False)
aT.Teste = aT.Teste.str.replace('(', '', regex=False)
aT.Teste = aT.Teste.str.replace('_', '', regex=False)
aT.Teste = aT.Teste.str.replace(';', '', regex=False)
aT.Teste = aT.Teste.str.replace(',', '', regex=False)
aT.Teste = aT.Teste.str.replace('&', '', regex=False)
aT.Teste = aT.Teste.str.replace('/', '', regex=False)
aT.Teste = aT.Teste.str.replace('#', '', regex=False)
aT.Teste = aT.Teste.str.replace('%', '', regex=False)
aT.Teste = aT.Teste.str.replace('$', '', regex=False)
aT.Teste = aT.Teste.str.replace("!", '', regex=False)
aT.Teste = aT.Teste.str.replace("?", '', regex=False)
aT.Teste = aT.Teste.str.replace("-", '', regex=False)
aT.Teste = aT.Teste.str.replace("+", '', regex=False)
aT.Teste = aT.Teste.str.replace("=", '', regex=False)
aT.Teste = aT.Teste.str.replace('rt', '', regex=False)
aT.Teste = aT.Teste.str.replace("\n", '', regex=False)

### Adicionando os tweets Treinamento em uma lista

Depois de limpar os tweets da tabela Treinamento, iremos adicioná-los em uma lista.

In [5]:
lista = at.groupby('Classificação').apply(lambda grupo: grupo.Treinamento.tolist()).tolist()

### Colocando os tweets da tabela Treinamento relevantes em uma lista

Os tweets da tabela Treinamento foram separados em relevantes e irrelevantes, dessa forma iremos colocar os tweets relevantes em uma lista, além disso iremos separar os tweets em cada palavra, para poder por fim criar um programa que faz a leitura de cada palavra.

In [6]:
lista_relevantes = []
lista_relevantes.append(lista[1])
lista_rel = []
for e in lista_relevantes[0]:
    strings = e.split(' ')
    for i in strings:
        if i not in lista_rel:
            lista_rel.append(i)
    lista_rel.append(strings)

### Adicionando os tweets da tabela Treianemento irrelevantes em uma lista

Os tweets irrelevantes agora, serão colocados em uma lista.

In [7]:
lista_irrelevantes  = []
lista_irrelevantes.append(lista[0])

### Calculando a probabilidade

Iremos separar o total relevantes (classificados como 1) e os irrelevantes (classificados como 0).

In [8]:
t0 = at.Classificação[at.Classificação == 0].count()
t1 = at.Classificação[at.Classificação == 1].count()

T = t0 + t1

p0 = t0 / T
p1 = t1 / T

print("Probabilidade de Irrelevante: {:.2f}%".format(p0*100))
print("Probabilidade de Relevante: {:.2f}%".format(p1*100))

Probabilidade de Irrelevante: 91.33%
Probabilidade de Relevante: 8.67%


Iremos criar um programa que percorrerá a lista total dos tweets limpos e depois cada palavra de cada tweet, para tirar o CAPSLOCK de todas as palavras, para ter uma melhor leitura.

In [9]:
prob_palavra = []

for tweet in lista:
    for palavra in tweet:
        if palavra.lower() not in prob_palavra:
            prob_palavra.append(palavra.lower())

Criaremos uma função que percorrerá a lista dos tweets relevantes e por fim irá calcular a probabilidade dos tweets relevantes. O mesmo faremos com os tweets irrelevantes, para descobrir a probabilidade.


In [10]:
def PR(palavra):
    evento = 1
    total = 0
    for tweet in lista_rel:
        total += len(tweet)
        
        for i in tweet:
            if i.lower() == palavra.lower():
                evento += 1
                
    return evento/(total + len(prob_palavra))

def PI(palavra):
    evento = 1
    total = 0
    for tweet in lista_irrelevantes:
        total += len(tweet)
        
        for i in tweet:
            if i.lower() == palavra.lower():
                evento += 1
    return evento/(total + len(prob_palavra))

In [11]:
def avaliando(lista):
    avaliacao = []

    for tweet in lista: 
        RELEVANTE = []
        IRRELEVANTE = []

        for palavra in tweet:
            RELEVANTE.append(PR(palavra))
            IRRELEVANTE.append(PI(palavra))

        tweet_relevante = 1
        tweet_irrelevante = 1

        for i in RELEVANTE:
            tweet_relevante *= i

        for i in IRRELEVANTE:
            tweet_irrelevante *= i

        PR_tweet = tweet_irrelevante * p1
        PI_tweet = tweet_relevante * p0

        if PR_tweet > PI_tweet:
            avaliacao.append('Relevante')
        else:
            avaliacao.append('Irrelevante')
    return avaliacao

In [18]:
lista_teste = []
for tweet in aT.Teste:
    lista_teste.append(tweet)

avaliacao = avaliando(lista_teste)

resposta_probabilidade = aT.rename(index=str, columns={'Unnamed: 1':'Relevancia'}).Relevancia

In [16]:
mt = 0
ma = 0
ga = 0
mi = 0



for i in range(len(resposta_probabilidade)):
    if resposta_probabilidade[i] == avaliacao[i] and avaliacao[i] == 'Relevante':
        mt += 1
    elif resposta_probabilidade[i] == avaliacao[i] and avaliacao[i] == 'Irrelevante':
        ma += 1
    elif resposta_probabilidade[i] != avaliacao[i] and avaliacao[i] == 'Relevante':
        ga += 1
    else:
        mi += 1

mt /= len(avaliacao)
ma /= len(avaliacao)
ga /= len(avaliacao)
mi /= len(avaliacao)

print(mt+ma+ga+mi)

print(mt)
print(ma)
print(mi)
print(ga)

len(avaliacao)

1.0
0.0
0.0
1.0
0.0


200

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
